In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler

### Read data and preprocess

In [5]:
df=pd.read_csv('churn_modeling.csv')
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
df[['Exited','CustomerId']].groupby('Exited').count()

,CustomerId
Exited,
0,7963
1,2037


In [8]:
neg, pos = np.bincount(df['Exited'])
total = neg + pos
print('Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Total: 10000
    Positive: 2037 (20.37% of total)



In [66]:
df['Gender']=df['Gender'].apply(lambda x:1 if x=='Male' else 0)

In [67]:
X,y=df.drop(['Exited','RowNumber','CustomerId','Surname','Geography'],1),df['Exited']

In [68]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [69]:
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)


### model 1

In [105]:
metrics = [  
    tf.metrics.BinaryAccuracy('Accuracy'),
    tf.metrics.Precision(name='Precision'),
    tf.metrics.Recall(name='Recall')]

model1=keras.models.Sequential()

model1.add(layers.BatchNormalization())
model1.add(layers.Dropout(0.3))
model1.add(layers.Dense(512, activation='relu', name='hidden_1'))
model1.add(layers.Dense(1, activation='sigmoid', name='output'))

model1.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=metrics)

In [187]:


checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model1.h5", save_best_only=True)

# Creat EarlyStopping checkpoint:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)


model1.fit(X_train, y_train,
                    epochs=150,
                     batch_size=1024,
                     validation_split=0.2,
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/150
7/7 [==============================] - 0s 18ms/step - loss: 0.3925 - Accuracy: 0.8356 - Precision: 0.7509 - Recall: 0.3056 - val_loss: 0.3535 - val_Accuracy: 0.8519 - val_Precision: 0.8829 - val_Recall: 0.3043
Epoch 2/150
7/7 [==============================] - 0s 13ms/step - loss: 0.3921 - Accuracy: 0.8364 - Precision: 0.7609 - Recall: 0.3033 - val_loss: 0.3512 - val_Accuracy: 0.8550 - val_Precision: 0.8689 - val_Recall: 0.3292
Epoch 3/150
7/7 [==============================] - 0s 11ms/step - loss: 0.3946 - Accuracy: 0.8367 - Precision: 0.7384 - Recall: 0.3245 - val_loss: 0.3535 - val_Accuracy: 0.8519 - val_Precision: 0.8632 - val_Recall: 0.3137
Epoch 4/150
7/7 [==============================] - 0s 9ms/step - loss: 0.3909 - Accuracy: 0.8356 - Precision: 0.7228 - Recall: 0.3313 - val_loss: 0.3548 - val_Accuracy: 0.8512 - val_Precision: 0.8750 - val_Recall: 0.3043
Epoch 5/150
7/7 [==============================] - 0s 11ms/step - loss: 0.3895 - Accuracy: 0.8392 - Precision: 0.

In [190]:
 eval=model1.evaluate(X_train, y_train, batch_size=1024, verbose=0)
 print('Training: Los= {:0.4f}, Accuracy= {:0.4f}, Precision= {:0.4f}, Recall={:0.4f}'.format(eval[0],eval[1],eval[2],eval[3]))

Training: Los= 0.3517, Accuracy= 0.8551, Precision= 0.8625, Recall=0.3510


In [194]:
eval=model1.evaluate(X_test, y_test, verbose=0)
print('Testing: Los= {:0.4f}, Accuracy= {:0.4f}, Precision= {:0.4f}, Recall={:0.4f}'.format(eval[0],eval[1],eval[2],eval[3]))

Training: Los= 0.3468, Accuracy= 0.8560, Precision= 0.8221, Recall=0.3410


### model 2
Try to add inital bias to output layer and weights to deal with imbalanced data

In [88]:
initial_bias = np.log([pos/neg])
output_bias = tf.keras.initializers.Constant(initial_bias)
initial_bias

array([-1.36332768])

In [162]:
weight_for_0 = (1 / neg) * (total / 8.0)
weight_for_1 =(1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
class_weight

{0: 0.15697601406505088, 1: 2.454590083456063}

In [183]:
model2=keras.models.Sequential([
      keras.layers.Dense(64, activation='relu',input_shape=(X_train.shape[-1],)),
       keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
      keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias),

])



In [184]:

metrics = [  
    tf.metrics.BinaryAccuracy('Accuracy'),
    tf.metrics.Precision(name='Precision'),
    tf.metrics.Recall(name='Recall')]


checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model2.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model2.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = metrics)
model2.fit(X_train, y_train,validation_split=0.2, batch_size = 1024, epochs = 150,callbacks=[checkpoint_cb,early_stopping_cb])  


Epoch 1/150
7/7 [==============================] - 1s 64ms/step - loss: 0.5291 - Accuracy: 0.7936 - Precision: 1.0000 - Recall: 7.5643e-04 - val_loss: 0.5002 - val_Accuracy: 0.7987 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 2/150
7/7 [==============================] - 0s 10ms/step - loss: 0.5001 - Accuracy: 0.7934 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - val_loss: 0.4827 - val_Accuracy: 0.7987 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 3/150
7/7 [==============================] - 0s 10ms/step - loss: 0.4848 - Accuracy: 0.7934 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - val_loss: 0.4684 - val_Accuracy: 0.7987 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00
Epoch 4/150
7/7 [==============================] - 0s 11ms/step - loss: 0.4708 - Accuracy: 0.7941 - Precision: 1.0000 - Recall: 0.0030 - val_loss: 0.4534 - val_Accuracy: 0.8025 - val_Precision: 1.0000 - val_Recall: 0.0186
Epoch 5/150
7/7 [==============================] - 0s 11ms/step - lo

In [195]:
eval=model2.evaluate(X_train, y_train, batch_size=1024, verbose=0)
print('Training: Los= {:0.4f}, Accuracy= {:0.4f}, Precision= {:0.4f}, Recall={:0.4f}'.format(eval[0],eval[1],eval[2],eval[3]))

Training: Los= 0.3211, Accuracy= 0.8687, Precision= 0.7845, Recall=0.4982


In [196]:
  
eval = model2.evaluate(X_test, y_test,verbose=0)  
print('Testing: Los= {:0.4f}, Accuracy= {:0.4f}, Precision= {:0.4f}, Recall={:0.4f}'.format(eval[0],eval[1],eval[2],eval[3]))

Testing: Los= 0.3398, Accuracy= 0.8590, Precision= 0.7110, Recall=0.4758
